In [11]:
import xarray as xr
import glob
import dask
from utils import geo
from dask.diagnostics import ProgressBar

In [9]:
rootdir = '/projects/SOCCOM/datasets/ESM2M_PPE/'
local = 'archive/kbr/fre/verona/ESM2M/20161113/ANALYSIS/CONTROL/CHL/'
filename = 'chl_*.nc'
lonname,latname = 'xt_ocean','yt_ocean'

print('Opening.')
paths = glob.glob(rootdir+local+filename)
with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ds = xr.open_mfdataset(paths,use_cftime=True)
ds = ds.sel({latname:slice(-80,-55),'st_ocean':5})
# Get area
ds,xgrid = geo.get_xgcm_horizontal(ds,axes_dims_dict={'X':lonname,'Y':latname},periodic='X')
ds['area'] = ds['dxC']*ds['dyC']
ds = ds.chunk({'time':60})
ds

Opening.


<xarray.Dataset>
Dimensions:         (time: 3600, yt_ocean: 25, xt_ocean: 360, yu_ocean: 200,
                     xu_ocean: 360, nv: 2, st_edges_ocean: 51,
                     xt_ocean_left: 360, yt_ocean_left: 25)
Coordinates: (12/14)
    geolat_c        (yu_ocean, xu_ocean) float32 dask.array<chunksize=(200, 360), meta=np.ndarray>
    geolat_t        (yt_ocean, xt_ocean) float32 dask.array<chunksize=(25, 360), meta=np.ndarray>
    geolon_c        (yu_ocean, xu_ocean) float32 dask.array<chunksize=(200, 360), meta=np.ndarray>
    geolon_t        (yt_ocean, xt_ocean) float32 dask.array<chunksize=(25, 360), meta=np.ndarray>
  * nv              (nv) float64 1.0 2.0
  * st_edges_ocean  (st_edges_ocean) float64 0.0 10.0 20.0 ... 5.133e+03 5.5e+03
    ...              ...
  * xt_ocean        (xt_ocean) float64 -279.5 -278.5 -277.5 ... 77.5 78.5 79.5
  * xu_ocean        (xu_ocean) float64 -279.0 -278.0 -277.0 ... 78.0 79.0 80.0
  * yt_ocean        (yt_ocean) float64 -79.5 -78.5 -77.5 ... -57.5 -56.5 -55.5
  * yu_ocean        (yu_ocean) float64 -81.0 -80.0 -79.0 ... 88.0 89.0 90.0
  * xt_ocean_left   (xt_ocean_left) float64 -280.0 -279.0 -278.0 ... 78.0 79.0
  * yt_ocean_left   (yt_ocean_left) float64 -80.0 -79.0 -78.0 ... -57.0 -56.0
Data variables:
    average_DT      (time) timedelta64[ns] dask.array<chunksize=(60,), meta=np.ndarray>
    average_T1      (time) object dask.array<chunksize=(60,), meta=np.ndarray>
    average_T2      (time) object dask.array<chunksize=(60,), meta=np.ndarray>
    chl             (time, yt_ocean, xt_ocean) float32 dask.array<chunksize=(60, 25, 360), meta=np.ndarray>
    time_bnds       (time, nv) timedelta64[ns] dask.array<chunksize=(60, 2), meta=np.ndarray>
    dxG             (xt_ocean_left, yt_ocean) float64 dask.array<chunksize=(360, 25), meta=np.ndarray>
    dyG             (xt_ocean, yt_ocean_left) float64 dask.array<chunksize=(360, 25), meta=np.ndarray>
    dxC             (xt_ocean, yt_ocean) float64 dask.array<chunksize=(360, 25), meta=np.ndarray>
    dyC             (xt_ocean, yt_ocean) float64 dask.array<chunksize=(360, 25), meta=np.ndarray>
    rC              (xt_ocean, yt_ocean) float64 dask.array<chunksize=(360, 25), meta=np.ndarray>
    area            (xt_ocean, yt_ocean) float64 dask.array<chunksize=(360, 25), meta=np.ndarray>
Attributes:
    filename:   ocean_topaz_tracers_month.000101-000512.chl.nc
    title:      ESM2M_pi-control_topaz_C3_verona_NEWDIAG_V4
    grid_type:  mosaic
    grid_tile:  1
    history:    Sun Jul 12 03:29:04 2020: ncks -d time,0,11 ocean_topaz_trace...
    NCO:        4.0.3

In [13]:
# Get monthly variance and monthly climatology
dsclim = ds['chl'].groupby('time.month').mean()
dsvar = ds['chl'].groupby('time.month').var()
dsperc = (dsvar/dsclim)*100

In [14]:
outfile = 'esm2m-control.monthly.percentagevariance.JANtoDEC.0001-0300.ForBuchovecky.nc'
with ProgressBar():
    dsperc.to_netcdf('../../../data/'+outfile)

[####################################### ] | 99% Completed | 57.50 ss

/home/graemem/.conda/envs/core/lib/python3.10/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed | 58.64 s
